In [1]:
import sys  ###resetmcl1
sys.path.append('../scripts/')
from mcl import *

In [2]:
class ResetMcl(Mcl): 
    def __init__(self, envmap, init_pose, num, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}, \
                 distance_dev_rate=0.14, direction_dev=0.05):
        super().__init__(envmap, init_pose, num, motion_noise_stds, distance_dev_rate, direction_dev)
        self.alphas = {}  #α値の記録（ランドマークの観測数ごと）
        
    def observation_update(self, observation):
        for p in self.particles:
            p.observation_update(observation, self.map, self.distance_dev_rate, self.direction_dev) 
        
        #alpha値の記録
        alpha = sum([p.weight for p in self.particles])
        obsnum = len(observation)
        if not obsnum in self.alphas: self.alphas[obsnum] = []
        self.alphas[obsnum].append(alpha)
            
        self.set_ml()
        self.resampling() #ここで重みの合計は1になる

In [3]:
def trial():
    time_interval = 0.1
    world = World(300, time_interval, debug=True) 

    ### 地図を生成して3つランドマークを追加 ###
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ### ロボットを作る ###
    initial_pose = np.array([0, 0, 0]).T
    pf = ResetMcl(m, initial_pose, 100)
    circling = EstimationAgent(time_interval, 0.2, 10.0/180*math.pi, pf)
    r = Robot(initial_pose, sensor=Camera(m), agent=circling, color="red")
    world.append(r)

    world.draw()
    
    return pf

pf = trial()

In [4]:
for num in pf.alphas: ###mclalpharesult
    print("landmarks:", num, "particles:", len(pf.particles), "min:", min(pf.alphas[num]), "max:", max(pf.alphas[num]))

landmarks: 2 particles: 100 min: 0.22390594484416315 max: 17.764815065661896
landmarks: 1 particles: 100 min: 0.04456137572738368 max: 13.004374720939975
landmarks: 0 particles: 100 min: 1.0000000000000007 max: 1.0000000000000007


In [5]:
def trial():
    time_interval = 0.1
    world = World(40, time_interval, debug=True) 

    ### 地図を生成して3つランドマークを追加 ###
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ### ロボットを作る ###
    pf = ResetMcl(m, np.array([-4, -4, 0]).T, 100)
    circling = EstimationAgent(time_interval, 0.2, 10.0/180*math.pi, pf)
    r = Robot(np.array([0,0,0]).T, sensor=Camera(m), agent=circling, color="red")
    world.append(r)

    world.draw()
    
    return pf
    
pf = trial()

In [6]:
for num in pf.alphas: ###mclalpharesult2
    print("landmarks:", num, "particles:", len(pf.particles), "min:", min(pf.alphas[num]), "max:", max(pf.alphas[num]))

landmarks: 2 particles: 100 min: 0.0 max: 1.3466655425634616e-93
landmarks: 1 particles: 100 min: 2.3373005364474248e-269 max: 5.793366598372977
landmarks: 0 particles: 100 min: 1.0000000000000007 max: 1.0000000000000007


In [7]:
pf.alphas

{2: [8.946435289371282e-144,
  1.3466655425634616e-93,
  1.1969463785125035e-97,
  6.420143544461477e-298,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  9.386105190351529e-301,
  1.534075061425e-312,
  3.1979511714764463e-296,
  2.6847285169515614e-276],
 1: [1.0463613821796277e-112,
  2.0063060607720784e-07,
  1.0488631743586148e-06,
  0.003245487078127109,
  4.4495288350830396e-05,
  0.00047654905457778743,
  4.892812482410206e-05,
  7.441507868232858e-05,
  0.0020697774326909895,
  0.006022909810809369,
  0.00146855371628662,
  0.00027443595901814545,
  0.0006345450339847864,
  0.0003666688417901743,
  0.0008940685335963873,
  0.00022966457451834005,
  0.00028330699885764963,
  0.0024594107205467134,
  0.0006087242133377529,
  0.00025195848767216626,
  0.00020979327531830993,
  0.00037815502867614856,
  0.0041507766803532175,
  0.0009885495474145389,
  0.00014552732012136546,
  0.000243341991704